In [1]:
import sys

# 动态添加test_package文件夹的路径，为了能让此文件夹下的
# 自定义包成功的导入
# 要根据你自己的实际包的模块来决定路径。


if sys.path[-1] != '../../../' :
    sys.path.append('../')
    sys.path.append('../../')
    sys.path.append('../../../')
else:
    pass

# 打印所有python解释器可以搜索到的所有路径
print(sys.path)

['', 'D:\\Python\\Anaconda3\\python36.zip', 'D:\\Python\\Anaconda3\\DLLs', 'D:\\Python\\Anaconda3\\lib', 'D:\\Python\\Anaconda3', 'C:\\Users\\Yukirito\\AppData\\Roaming\\Python\\Python36\\site-packages', 'D:\\Python\\Anaconda3\\lib\\site-packages', 'D:\\Python\\Anaconda3\\lib\\site-packages\\win32', 'D:\\Python\\Anaconda3\\lib\\site-packages\\win32\\lib', 'D:\\Python\\Anaconda3\\lib\\site-packages\\Pythonwin', 'D:\\Python\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Yukirito\\.ipython', '../', '../../', '../../../']


### <font size=4>网格搜索</font>

In [2]:
import numpy as np
from sklearn import datasets

In [3]:
digits = datasets.load_digits()

In [4]:
X = digits.data
y = digits.target

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

sk_knn_clf = KNeighborsClassifier(n_neighbors=4, weights='uniform')
sk_knn_clf.fit(X_train, y_train)
sk_knn_clf.score(X_test, y_test)

0.9916666666666667

### GridSearch

In [7]:
'''
使用 Grid Search 之前, 首先我们要定义自己的参数, 在这里是 parameter grid

parameter grid 是一个数组, 
    对于数组中每一个元素, 是一个字典
    每一个字典代表我们要进行的一组网格搜索
    每一组网格搜索, 里面的 key 对应的是要遍历的 取值范围(写在数组里面)
    
    eg.
        首先看这里的第一组
            对于这个字典, 它的键对应的就是参数的名称, 他的 值 是一个列表, 在列表中存储的是 键 对应的 参数 的所有的可能的范围
            在这第一组网格搜索字典中, 键 'weights'对应的列表 ['uniform'] 只取一个取值范围; 相应的 n_neighbors 值我们只取 1~10 共十个值
            以上就是我们要搜索的第一组参数
        
        我们我要搜索的第二组参数
            对应 'weights' 只取一个值, 就是 ['distance']
            对应 'n_neighbors' 也是取 1~10
            对应 'p', 是明可夫斯基距离中的 p, 我们取 1~5
        
        这样一来我们就定义了我们要搜索的所有参数的集合
        与此同时是我们可以发现 参数p 对 weights='distance' 这样一个依赖, 被我们定义的参数集合中, 放在同一个字典中来表示
        也就是当 weights='uniform' 的时候, 就不会对 p 进行搜索

'''
param_grid = [{
    'weights': ['uniform'],
    'n_neighbors': [i for i in range(1, 11)]
}, {
    'weights': ['distance'],
    'n_neighbors': [i for i in range(1, 11)],
    'p': [i for i in range(1, 6)]
}]

In [8]:
'''
我们定义好了所有参数集合之后, 下一步就是创建一个我们要进行 网格搜索 对应的机器学习算法

在这里我们要对 kNN 进行网格搜索

先创建一个默认的 knn_clf

不需要传任何的参数, 因为等会要用 grid search 选择出最佳参数
'''
knn_clf = KNeighborsClassifier()

In [9]:
'''
加下来调用 scikit-learn 中 网格搜索 的方法

这个方法在 model_selection 中, 回忆一下, train_test_split 方法也是在这个模块中
因为 网格搜索 也是为了寻找到更好的模型

impoet GridSearchCV

CV 是什么? 我们在后续会具体讲到

然后开始创建 GridSearchCV 对应的实例对象 grid_search
    
    grid_search = GridSearchCV()
    
        传入两个参数
            参数一: 算法分类器对象, 例如 knn_clf
            参数二: 网格搜索对应的参数 -> param_grid
    这样我们就对应好了网格搜索相应的对象

'''

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(knn_clf, param_grid)

In [10]:
'''
这个过程就是基于X_train, y_train训练数据集
针对我们定义的所有参数来尝试寻找最佳的模型

这个过程会相对比较慢, 可以记录一下时间
'''

'\n这个过程就是基于X_train, y_train训练数据集\n针对我们定义的所有参数来尝试寻找最佳的模型\n\n这个过程会相对比较慢, 可以记录一下时间\n'

In [11]:
%%time
grid_search.fit(X_train, y_train)

Wall time: 4min 13s


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
'''
运行完 网格搜索对象训练 之后, 我们就可以调用对象里的属性
我们可以发现这些属性参数名后面都跟着下划线, 
    例如 best_estimator_ -> 不是用户传入的参数, 而是根据用户传入的参数, 我们这个类自己计算出来的结果, 那么命名规范就是这样


grid_search.best_estimator_
    返回 网格搜索 搜索到的最佳的分类器
    
    weights='distance',p=3,...与之前我们取到的数值不同
    在上面我们在train_test_split中random_state=666已经统一了,
    而 网格搜索 评价我们分类器的准确度的方式是更加复杂的一种方式, 也就是 CV -> Cross Validation - 交叉验证, 在后面会详细介绍
    交叉验证的准确度是更加准确的
    
'''
# 返回 网格搜索 搜索到的最佳的分类器对应的参数
grid_search.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=3,
           weights='distance')

In [17]:
'''
返回 网格搜索 搜索到的最佳的分类器对应的准确度

这个准确度是 0.9856..., 比 4.5节 我们得到的数值要低一些, 似乎我们的准确度并不是那么好

其实是因为我们的评判标准不一样
在机器学习算法领域, 如何评判算法的好坏是一个非常重要的课题
我们后续会进行详细介绍
'''

grid_search.best_score_

0.9853862212943633

In [18]:
'''
返回 网格搜索 搜索到的最佳的分类器对应的参数

'''
grid_search.best_params_

{'n_neighbors': 3, 'p': 3, 'weights': 'distance'}

In [19]:
'''
获取 网格搜索 得到的最佳分类器
'''
knn_clf = grid_search.best_estimator_

In [22]:
'''
预测结果
'''

knn_clf.predict(X_test)

array([8, 1, 3, 4, 4, 0, 7, 0, 8, 0, 4, 6, 1, 1, 2, 0, 1, 6, 7, 3, 3, 6,
       5, 2, 9, 4, 0, 2, 0, 3, 0, 8, 7, 2, 3, 5, 1, 3, 1, 5, 8, 6, 2, 6,
       3, 1, 3, 0, 0, 4, 9, 9, 2, 8, 7, 0, 5, 4, 0, 9, 5, 5, 8, 7, 4, 2,
       8, 8, 7, 5, 4, 3, 0, 2, 7, 2, 1, 2, 4, 0, 9, 0, 6, 6, 2, 0, 0, 5,
       4, 4, 3, 1, 3, 8, 6, 4, 4, 7, 5, 6, 8, 4, 8, 4, 6, 9, 7, 7, 0, 8,
       8, 3, 9, 7, 1, 8, 4, 2, 7, 0, 0, 4, 9, 6, 7, 3, 4, 6, 4, 8, 4, 7,
       2, 6, 9, 5, 8, 7, 2, 5, 5, 9, 7, 9, 3, 1, 9, 4, 4, 1, 5, 1, 6, 4,
       4, 8, 1, 6, 2, 5, 2, 1, 4, 4, 3, 9, 4, 0, 6, 0, 8, 3, 8, 7, 3, 0,
       3, 0, 5, 9, 2, 7, 1, 8, 1, 4, 3, 3, 7, 8, 2, 7, 2, 2, 8, 0, 5, 7,
       6, 7, 3, 4, 7, 1, 7, 0, 9, 2, 8, 9, 3, 8, 9, 1, 1, 1, 9, 8, 8, 0,
       3, 7, 3, 3, 4, 8, 2, 1, 8, 6, 0, 1, 7, 7, 5, 8, 3, 8, 7, 6, 8, 4,
       2, 6, 2, 3, 7, 4, 9, 3, 5, 0, 6, 3, 8, 3, 3, 1, 4, 5, 3, 2, 5, 6,
       9, 6, 9, 5, 5, 3, 6, 5, 9, 3, 7, 7, 0, 2, 4, 9, 9, 9, 2, 5, 6, 1,
       9, 6, 9, 7, 7, 4, 5, 0, 0, 5, 3, 8, 4, 4, 3,

In [25]:
'''
准确率
'''
knn_clf.score(X_test, y_test)

0.9833333333333333

以上过程就是使用 scikit-learn 中 网格搜索 寻找最佳超参数的方式

<br/>

<br/>

'''
这里 GridSearchCV 还可以传入更多的参数来帮助我们更好地了解这个类

首先是
    n_jobs
    
    GridSearchCV就是根据我们定义的 param_grid 参数数组来创建很多分类器, 然后比较这些分类器谁更好而已
    那么我们创建很多分类器这个过程, 显然是可以 并行处理 的
    
    n_jobs 就决定了为你的计算机分配几个 核core 来进行这种并行处理的过程
    
    n_jobs默认为1, 就是计算机默认使用单核来进行分类器处理
    如果你的计算机是多核的, 有两个核可以传 n_jobs=2, 有四个核可以传 n_jobs=4或3或2
    当然也可以传 n_jobs=-1, 也就是说我不去考虑我的计算机由多少核,全都用于这个网格搜索
    
    在这里我们尝试一下传入 n_jobs=-1

    我们这里依然查看一下代码运行时间的变化
    
另外还有一个参数
    
    verbose：日志冗长度，int：冗长度，0：不输出训练过程，1：偶尔输出，>1：对每个子模型都输出。

    verbose=2
    
    在搜索的时候进行一些输出
    
    在上一步我们训练 grid_search 的时候, 期间是没有输出信息的
    这个参数的使用, 可以帮助我们看到搜索每一个参数的时候的输出信息, 帮助我们了解这个搜索的过程
    尤其当我们在一个大的项目里面, 我们是需要看到搜索过程的
    
    比如一个复杂的算法, 一个搜索很有可能要运行几个小时, 在这种情况下, 我们有输出能帮助我们及时了解现在的搜索状态, 是非常重要的
    
    verbose 传入的是整数, 传入的数值越大, 输出的信息就越详细
        可以试一下不同的值对应的信息是怎样的

'''

In [29]:
%%time
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs=-1, verbose=10)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   54.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.3min finished


Wall time: 1min 17s


Fitting 3 folds for each of 60 candidates, totalling 180 fits

folds 是 交叉验证 中的一个概念, 在后面会加以讲解

60 candidates
    代表我们要搜索60组参数 10 + 50
    
因为本次 n_jobs=-1, 所以速度快了1倍不止

我们是用GridSearchCV就是为了对不同的超参数进行搜索

### 更多的距离定义

事实上, kNN算法还有更多的超参数

    对于明可夫斯基距离, 我们已经有了超参数 p
    
    在统计学上, 我们可以通过两个样本间的相似度来定义超参数, 而相似度又有很多定义的方法
    
    比如:
        * 向量空间余弦相似度 Cosine Similarity
        * 调整余弦相似度 Adjusted Cosine Similarity
        * 皮尔森相关系数 Pearson Correlation Coefficient
        * Jaccard相似系数 Jaccard Coefficient
        
    对于这些相似系数的定义如何, 对于这样的数学定义背后, 最佳的应用场景是怎么样的, 在这里不做多详细的介绍, 仅供拓展
    
    通过这个例子我们可以了解为什么在使用机器学习算法的时候, 数学知识是非常重要的, 但这里是机器学习的基础笔记, 不会深究诸多数学上的定义
    
在 knn_clf 官方文档中, 有 metric 参数, 默认是 minkowski

而我们可以看到minkowski是由DistanceMetric class定义的

[sklearn.neighbor.DistanceMetric](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric)

我们可以发现, Metric 其实也是一个超参数, 所以我们也可以用网格搜索的方法将Metric也加入进参数中

有兴趣的话, 也可以尝试从底层实现网格搜索